In [3]:
!nvidia-smi

Wed Oct 16 19:51:04 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1080      WDDM  |   00000000:01:00.0  On |                  N/A |
| 21%   48C    P8             18W /  200W |     532MiB /   8192MiB |      4%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!git clone https://github.com/rslim087a/track

Cloning into 'track'...
Updating files:  22% (2687/12160)
Updating files:  23% (2797/12160)
Updating files:  24% (2919/12160)
Updating files:  25% (3040/12160)
Updating files:  26% (3162/12160)
Updating files:  27% (3284/12160)
Updating files:  28% (3405/12160)
Updating files:  29% (3527/12160)
Updating files:  30% (3648/12160)
Updating files:  31% (3770/12160)
Updating files:  32% (3892/12160)
Updating files:  33% (4013/12160)
Updating files:  34% (4135/12160)
Updating files:  35% (4256/12160)
Updating files:  36% (4378/12160)
Updating files:  37% (4500/12160)
Updating files:  38% (4621/12160)
Updating files:  39% (4743/12160)
Updating files:  40% (4864/12160)
Updating files:  41% (4986/12160)
Updating files:  42% (5108/12160)
Updating files:  43% (5229/12160)
Updating files:  44% (5351/12160)
Updating files:  45% (5472/12160)
Updating files:  45% (5561/12160)
Updating files:  46% (5594/12160)
Updating files:  47% (5716/12160)
Updating files:  48% (5837/12160)
Updating files:  49% (59

In [4]:
!pip install tensorflow imgaug

INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached networkx-3.1-py3-none-any.whl.metadata (5.3 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/276.5 MB ? eta -:--:--
   ---------------------------------------- 0.5/276.5 MB 5.6 MB/s eta 0:00:50
    --------------------------------------- 4.5/276.5 MB 15.8 MB/s eta 0:00:18
   - -------------------------------------- 8.7/276.5 MB 17.3 MB/s eta 0:00:16
   - -------------------------------------- 12.6/276.5 MB 17.5 MB/s eta 0:00:16
   -- ------------------------------------- 19.1/276.5 MB 20.5 MB/s eta 0:00:13
   --- ------------------------------------ 26.2/276.5 MB 23.1 MB/s eta 0:00:11
   ---- --------------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.24.1 requires google-auth<2.0dev,>=1.21.1, but you have google-auth 2.35.0 which is incompatible.
google-cloud-storage 1.35.0 requires google-auth<2.0dev,>=1.11.0, but you have google-auth 2.35.0 which is incompatible.
llama-stack 0.0.1a5 requires typing-extensions<5,>=4.7, but you have typing-extensions 4.5.0 which is incompatible.
pydantic 2.9.2 requires typing-extensions>=4.6.1; python_version < "3.13", but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.23.4 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.


In [7]:
!pip install scikit-learn


  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.3 MB 5.6 MB/s eta 0:00:02
   --------------------------- ------------ 6.3/9.3 MB 20.3 MB/s eta 0:00:01
   ---------------------------------------  9.2/9.3 MB 18.4 MB/s eta 0:00:01
   ---------------------------------------- 9.3/9.3 MB 16.9 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)


In [18]:
import os 
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg

from tensorflow import keras 
import tensorflow as tf 
import cv2 
import pandas as pd 
import random 
import ntpath
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa


In [9]:
datadir = 'track'

In [10]:
# assign correct column name to dataset csv

columns = ['center','left', 'right','steering','throttle','reversed','speed' ]


In [11]:
data=pd.read_csv(os.path.join(datadir, "driving_log.csv"),names=columns)
#pd.set_option('display.max_colwidth',-1)

In [13]:
data.tail(10)

,center,left,right,steering,throttle,reversed,speed
4043,C:\Users\Amer\Desktop\new_track\IMG\center_201...,C:\Users\Amer\Desktop\new_track\IMG\left_2018_...,C:\Users\Amer\Desktop\new_track\IMG\right_2018...,0.0,0.0,0.0,26.97956
4044,C:\Users\Amer\Desktop\new_track\IMG\center_201...,C:\Users\Amer\Desktop\new_track\IMG\left_2018_...,C:\Users\Amer\Desktop\new_track\IMG\right_2018...,0.0,0.0,0.0,26.76278
4045,C:\Users\Amer\Desktop\new_track\IMG\center_201...,C:\Users\Amer\Desktop\new_track\IMG\left_2018_...,C:\Users\Amer\Desktop\new_track\IMG\right_2018...,0.0,0.0,0.0,26.60134
4046,C:\Users\Amer\Desktop\new_track\IMG\center_201...,C:\Users\Amer\Desktop\new_track\IMG\left_2018_...,C:\Users\Amer\Desktop\new_track\IMG\right_2018...,0.0,0.0,0.0,26.38760
4047,C:\Users\Amer\Desktop\new_track\IMG\center_201...,C:\Users\Amer\Desktop\new_track\IMG\left_2018_...,C:\Users\Amer\Desktop\new_track\IMG\right_2018...,0.0,0.0,0.0,26.22842
4048,C:\Users\Amer\Desktop\new_track\IMG\center_201...,C:\Users\Amer\Desktop\new_track\IMG\left_2018_...,C:\Users\Amer\Desktop\new_track\IMG\right_2018...,0.0,0.0,0.0,26.01768
4049,C:\Users\Amer\Desktop\new_track\IMG\center_201...,C:\Users\Amer\Desktop\new_track\IMG\left_2018_...,C:\Users\Amer\Desktop\new_track\IMG\right_2018...,0.0,0.0,0.0,25.80864
4050,C:\Users\Amer\Desktop\new_track\IMG\center_201...,C:\Users\Amer\Desktop\new_track\IMG\left_2018_...,C:\Users\Amer\Desktop\new_track\IMG\right_2018...,0.0,0.0,0.0,25.65296
4051,C:\Users\Amer\Desktop\new_track\IMG\center_201...,C:\Users\Amer\Desktop\new_track\IMG\left_2018_...,C:\Users\Amer\Desktop\new_track\IMG\right_2018...,0.0,0.0,0.0,25.44685
4052,C:\Users\Amer\Desktop\new_track\IMG\center_201...,C:\Users\Amer\Desktop\new_track\IMG\left_2018_...,C:\Users\Amer\Desktop\new_track\IMG\right_2018...,0.0,0.0,0.0,17.00781


In [14]:
def path_leaf(path):
    haed, tail =ntpath.split(path)
    return tail

In [15]:
data['center']= data['center'].apply(path_leaf)
data['left']= data['left'].apply(path_leaf)
data['right']= data['right'].apply(path_leaf)

In [16]:
data.head(10)

,center,left,right,steering,throttle,reversed,speed
0,center_2018_07_16_17_11_43_382.jpg,left_2018_07_16_17_11_43_382.jpg,right_2018_07_16_17_11_43_382.jpg,0.0,0.000000,0.0,0.649786
1,center_2018_07_16_17_11_43_670.jpg,left_2018_07_16_17_11_43_670.jpg,right_2018_07_16_17_11_43_670.jpg,0.0,0.000000,0.0,0.627942
2,center_2018_07_16_17_11_43_724.jpg,left_2018_07_16_17_11_43_724.jpg,right_2018_07_16_17_11_43_724.jpg,0.0,0.000000,0.0,0.622910
3,center_2018_07_16_17_11_43_792.jpg,left_2018_07_16_17_11_43_792.jpg,right_2018_07_16_17_11_43_792.jpg,0.0,0.000000,0.0,0.619162
4,center_2018_07_16_17_11_43_860.jpg,left_2018_07_16_17_11_43_860.jpg,right_2018_07_16_17_11_43_860.jpg,0.0,0.000000,0.0,0.615438
5,center_2018_07_16_17_11_43_930.jpg,left_2018_07_16_17_11_43_930.jpg,right_2018_07_16_17_11_43_930.jpg,0.0,0.000000,0.0,0.610506
6,center_2018_07_16_17_11_43_998.jpg,left_2018_07_16_17_11_43_998.jpg,right_2018_07_16_17_11_43_998.jpg,0.0,0.000000,0.0,0.606834
7,center_2018_07_16_17_11_44_069.jpg,left_2018_07_16_17_11_44_069.jpg,right_2018_07_16_17_11_44_069.jpg,0.0,0.000000,0.0,0.601971
8,center_2018_07_16_17_11_44_137.jpg,left_2018_07_16_17_11_44_137.jpg,right_2018_07_16_17_11_44_137.jpg,0.0,0.000000,0.0,0.598350
9,center_2018_07_16_17_11_44_209.jpg,left_2018_07_16_17_11_44_209.jpg,right_2018_07_16_17_11_44_209.jpg,0.0,0.024006,0.0,0.620654


In [19]:
# visualize the data 
num_bins = 25
hist, bins = np.histogram(data['steering'],num_bins)
center = (bins[:-1]+bins[1:0])*0.5
plt.bar(center, hist, width=0.05)

ValueError: operands could not be broadcast together with shapes (25,) (0,) 